In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest
from string import punctuation

In [3]:
import json
import csv

In [4]:
from gensim.summarization import keywords 

In [5]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shivani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shivani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [8]:
import requests

In [9]:
url = "https://www.codementor.io/hirenpatel545/5-best-javascript-web-scraping-libraries-and-tools-sicow2rx9"
headers = {'User-Agent':'Mozilla/5.0'}
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")

In [10]:
title = soup.find('title').text
print(title)

5 Best JavaScript Web Scraping Libraries and Tools | Codementor


In [11]:
all_headings = soup.find_all("h2")
# print(all_headings)
headings = []
for heading in all_headings:
    headings.append(heading.text)
    print(heading.text)
    

 Introduction
 1. Request (Aka HTTP Client) :
 2. Cheerio (Aka Parser) :
 3. Osmosis (Aka Parser) :
 4. Puppeteer (Aka Headless Chrome Browser for Automation) :
 5. Apify SDK (Aka The Complete Web Scraping Framework) :
 Conclusion


In [12]:
all_paragraphs = soup.find_all("p")

In [13]:
content = []

In [14]:
for x in all_paragraphs:
    content.append(x.get_text())

print(content[0])

As it’s obvious, Internet is getting overloaded with information and data. With the growth of data on the web, web scraping is also likely to become more and more important for businesses for mining the Internet for actionable insights. While there are various tools available for web scraping, a growing number of people spend their valuable time exploring web scraping libraries and tools for JavaScript.


In [15]:
def findSummary(content):
    tokens = word_tokenize(content.lower())
    sentences = sent_tokenize(content)
    stop_words = set(stopwords.words('english') + list(punctuation))
    words = [word for word in tokens if (word.isalpha() and word not in stop_words)]
#     words[:10]
    word_freq = FreqDist(words)
    rankings = defaultdict(int)
    for i, sentence in enumerate(sentences):
        for word in word_tokenize(sentence.lower()):
            if word in word_freq:
                rankings[i] += word_freq[word]
#      rankings    
    indexes = nlargest(3, rankings, key=rankings.get)
    final_sentences = [sentences[j] for j in sorted(indexes)]
    summary = ' '.join(final_sentences)
    return summary

In [16]:
summary = []

In [17]:
for para in all_paragraphs:
    summary_individual = findSummary(para.get_text())
    summary.append(summary_individual)

print(summary[0])

As it’s obvious, Internet is getting overloaded with information and data. With the growth of data on the web, web scraping is also likely to become more and more important for businesses for mining the Internet for actionable insights. While there are various tools available for web scraping, a growing number of people spend their valuable time exploring web scraping libraries and tools for JavaScript.


In [18]:
def createJSON(content,headings):
    print("Creating JSON object for each url")
    lang = "en"
    sentences = []
    tokens_sentences = []
    POS_tokens = []
    tokens = []
    tokens_sentences_lemmatized = []
    i = 0
    for c in content:
        tokens.append(keywords(c).split('\n'))
        sentences_per_para = sent_tokenize(c)
        tokens_sentences_per_para = []
        POS_tokens_per_para = []
        tokens_sentences_lemmatized_per_para = []
        for sentence in sent_tokenize(c):
            sent_lem = []
            words = word_tokenize(sentence)
            tokens_sentences_per_para.append(words)
            for word in words:
                lemm = lemmatizer.lemmatize(word)
                sent_lem.append(lemm)
            tokens_sentences_lemmatized_per_para.append(sent_lem)
            POS_tokens_per_para.append(nltk.pos_tag(words))
        sentences.append(sentences_per_para)
        tokens_sentences.append(tokens_sentences_per_para)
        POS_tokens.append(POS_tokens_per_para)
        tokens_sentences_lemmatized.append(tokens_sentences_lemmatized_per_para)
        
    dict = {
        "articles": headings,
        "lang":"en",
        "sentences":sentences,
        "tokens_sentences":tokens_sentences,
        "POS_tokens":POS_tokens,
        "tokens":tokens,
        "tokens_sentences_lemmatized":tokens_sentences_lemmatized
    }
    
    return dict;
    
#     JSON object creation   

In [19]:
urls = [ "https://www.codementor.io/hirenpatel545/5-best-javascript-web-scraping-libraries-and-tools-sicow2rx9","https://medium.com/ninjaconcept/interactive-dynamic-force-directed-graphs-with-d3-da720c6d7811","https://en.wikipedia.org/wiki/Alphabet_Inc."]
data = []
for url in urls:
    headers = {'User-Agent':'Mozilla/5.0'}
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    all_headings = soup.find_all("h2")
    all_paragraphs = soup.find_all("p")
    content = []
    for x in all_paragraphs:
        content.append(x.get_text())
        
    headings = []
    for heading in all_headings:
        headings.append(heading.text)
    
    dict = createJSON(content,headings)
    data.append(dict)

# print(data[0])
json_data = {"para":data}  
json_string = json.dumps(json_data)

Creating JSON object for each url
Creating JSON object for each url
Creating JSON object for each url


In [23]:
def mapJSONtoCSV(json_data):
    json_parsed = json.loads(json_data)
    j_data = json_parsed['para']
    print(j_data[1])
    csv_data = open('Data.csv', 'w')
    csvwriter = csv.writer(csv_data)
    count = 0
    for d in j_data:
        if count == 0:
            header = d.keys()
            csvwriter.writerow(header)
            count += 1
        csvwriter.writerow(d.values())
    csv_data.close()

In [24]:
print(mapJSONtoCSV(json_string))

{'articles': ['A guided tour through D3 version 4.0 force simulations with dynamic data and user interaction'], 'lang': 'en', 'sentences': [['Many companies have a tremendous amount of data which is not just hard to maintain and manage, but also hard to understand and analyse.', 'Here, data visualisation should find a remedy.', 'The main goal is to communicate information clearly and efficiently which improves the overall experience.'], ['In this post, I want to explain how we, at NinjaConcept, used D3 to create an interactive and dynamic force-directed graph which helped us to display nested data with complex relations in a simple natural way.'], ['Before we start, I want to explain which requirements we had to fulfill.', 'This might help to understand our decisions better.'], ['First of all, the data we had to display was composed of both specially prepared data as well as extractions of the companies long-standing knowledge management system(s).The hardware requirement was very inte

UnicodeEncodeError: 'charmap' codec can't encode character '\u2011' in position 4800: character maps to <undefined>